In [ ]:
! unzip merged_clean.txt.zip

Archive:  merged_clean.txt.zip
replace merged_clean.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: merged_clean.txt        
  inflating: __MACOSX/._merged_clean.txt  


In [ ]:
! pip install hmmlearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 KB 13.5 MB/s eta 0:00:00


In [ ]:
! head merged_clean.txt


Lovely Ilonka

There was once a king's son who told his father that he wished to marry.

'No, no!' said the king; 'you must not be in such a hurry. Wait till you
have done some great deed. My father did not let me marry till I had won
the golden sword you see me wear.'

The prince was much disappointed, but he never dreamed of disobeying his


# Hidden Markov Model

In [ ]:
import sys, json, codecs, pickle, argparse
import numpy as np
import string

import joblib
from sklearn.preprocessing import LabelEncoder
from hmmlearn import hmm
from nltk import FreqDist

In [ ]:
file = open("./merged_clean.txt", 'r', encoding='utf-8').read()
sentences = file.split("\n")
table = str.maketrans('', '', string.punctuation)

for index, sentence in enumerate(sentences):
  if sentence == "":
      del sentences[index]

In [ ]:
lines = [line.split() for line in sentences]

for line_index, line in enumerate(lines):
  for word_index, word in enumerate(line):
    lines[line_index][word_index] = word.translate(table)

words = [word.lower() for line in lines for word in line]

table = str.maketrans('', '', string.punctuation)
for index, word in enumerate(words):
    words[index] = word.translate(table)

alphabet = set(words)
le = LabelEncoder()
le.fit(list(alphabet))

seq = le.transform(words)
features = np.fromiter(seq, np.int64)
features = features.reshape(-1, 1)
# features = np.atleast_2d(features).T
fd = FreqDist(seq)

In [ ]:
print(len(alphabet))
print(words[:10])
print(seq[:10])
print(features[:10])
print(fd.get(0))

39738
['lovely', 'ilonka', 'there', 'was', 'once', 'a', 'kings', 'son', 'who', 'told']
[21341 17993 35093 38019 24826   574 19881 32604 38667 35713]
[[21341]
 [17993]
 [35093]
 [38019]
 [24826]
 [  574]
 [19881]
 [32604]
 [38667]
 [35713]]
225


In [ ]:
model = hmm.CategoricalHMM(n_components=3, init_params="ste")

In [ ]:
print(lines[:2])
print(features[:2])

[['Lovely', 'Ilonka'], ['There', 'was', 'once', 'a', 'kings', 'son', 'who', 'told', 'his', 'father', 'that', 'he', 'wished', 'to', 'marry']]
[[21341]
 [17993]]


In [ ]:
model = model.fit(features)

In [ ]:
symbols, _states = model.sample(20, random_state=42)
output = le.inverse_transform(np.squeeze(symbols))
for word in output:
        print(word, end=" ")

who never carry to stable with earrings country of get before his the my and come while that and know 

# GPT-2

In [ ]:
! pip install --force-reinstall transformers[tf-cpu] tensorflow==2.8.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached transformers-4.26.1-py3-none-any.whl (6.3 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 497.6/497.6 MB 3.0 MB/s eta 0:00:00
  Using cached absl_py-1.4.0-py3-none-any.whl (126 kB)
  Using cached h5py-3.8.0-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (4.7 MB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached grpcio-1.51.3-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (4.8 MB)
  Using cached setuptools-67.4.0-py3-none-any.whl (1.1 MB)
  Using cached wrapt-1.15.0-cp38-cp38-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (81 kB)
  Using cached termcolor-2.2.0-py3-none-any.whl (6.6 kB)
  Using cached tf_estimator_nightly-2.8.0.dev2021122109-py2.py3-none-any.whl (462 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached numpy-1.24.2-cp38-cp38-manylinux_2_17_x86_64.manyl

In [ ]:
from transformers import (
    GPT2Tokenizer,
    DataCollatorForLanguageModeling,
    TextDataset,
    GPT2LMHeadModel,
    TrainingArguments,
    Trainer,
    pipeline)

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2") 

loading file vocab.json from cache at /root/.cache/huggingface/hub/models--gpt2/snapshots/e7da7f221d5bf496a48136c0cd264e630fe9fcc8/vocab.json
loading file merges.txt from cache at /root/.cache/huggingface/hub/models--gpt2/snapshots/e7da7f221d5bf496a48136c0cd264e630fe9fcc8/merges.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at None
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--gpt2/snapshots/e7da7f221d5bf496a48136c0cd264e630fe9fcc8/config.json
Model config GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12

In [ ]:
print('vocabulary size: %d, max squence length: %d' % (tokenizer.vocab_size, tokenizer.model_max_length))
print('tokenize sequence "Once upon a time in a little village":', tokenizer('Once upon a time in a little village'))

vocabulary size: 50257, max squence length: 1024
tokenize sequence "Once upon a time in a little village": {'input_ids': [7454, 2402, 257, 640, 287, 257, 1310, 7404], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}


In [ ]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [ ]:
train_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path="merged_clean.txt",
    block_size=128)
     
test_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path="merged_clean.txt",
    block_size=128)

/usr/local/lib/python3.8/dist-packages/transformers/data/datasets/language_modeling.py:54: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(
Loading features from cached file cached_lm_GPT2Tokenizer_128_merged_clean.txt [took 0.003 s]
Loading features from cached file cached_lm_GPT2Tokenizer_128_merged_clean.txt [took 0.002 s]


In [ ]:
print(tokenizer.decode(train_dataset[5]))

 questioned the magpies the crippled one was the only one who knew where the three bulrushes were.


Then the prince started off with the lame magpie. They went on and on till they reached a great stone wall, many, many feet high.


‘Now, prince,’ said the magpie, ‘the three bulrushes are behind that wall.’


The prince wasted no time. He set his horse at the wall and leaped over it. Then he looked about for the three bulrushes, pulled them up and set off with them on his way home.


In [ ]:
model = GPT2LMHeadModel.from_pretrained('gpt2')

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--gpt2/snapshots/e7da7f221d5bf496a48136c0cd264e630fe9fcc8/config.json
Model config GPT2Config {
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "transformers_version": "4.26.1",
  "use_cach

In [ ]:
training_args = TrainingArguments(
    output_dir = 'data/out', # the output directory for the model predictions and checkpoints
    overwrite_output_dir = True, # overwrite the content of the output directory
    per_device_train_batch_size = 4, # the batch size for training
    per_device_eval_batch_size = 4, # the batch size for evaluation
    learning_rate = 5e-5, # defaults to 5e-5
    num_train_epochs = 1, # total number of training epochs to perform
)

trainer = Trainer(
    model = model,
    args = training_args,
    data_collator=data_collator,
    train_dataset = train_dataset,
    eval_dataset = test_dataset
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
import torch

torch.cuda.empty_cache()

In [ ]:
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 629
  Num Epochs = 1
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 158
  Number of trainable parameters = 124439808


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=158, training_loss=3.22121130062055, metrics={'train_runtime': 36.7198, 'train_samples_per_second': 17.13, 'train_steps_per_second': 4.303, 'total_flos': 41088172032000.0, 'train_loss': 3.22121130062055, 'epoch': 1.0})

In [ ]:
import time
time.sleep(3600)

In [ ]:
trainer.save_model()

Saving model checkpoint to data/out
Configuration saved in data/out/config.json
Configuration saved in data/out/generation_config.json
Model weights saved in data/out/pytorch_model.bin


In [ ]:
generator = pipeline('text-generation', tokenizer='gpt2', model='data/out')

loading configuration file data/out/config.json
Model config GPT2Config {
  "_name_or_path": "data/out",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "torch_dtype": "float32",
  "transformers_version": "4.26.1",
  "use_cache": true,
  "vocab_size": 50257
}

loading configur

In [ ]:
print(generator('There is a building', max_length=500)[0]['generated_text'])

Generate config GenerationConfig {
  "bos_token_id": 50256,
  "do_sample": true,
  "eos_token_id": 50256,
  "max_length": 50,
  "transformers_version": "4.26.1"
}

/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1186: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


There is a building that is as beautiful as heaven and as shining as every sky and a shining throne that has ever been built in this earth, and it stands for the first time, but to tell you of its importance would require a thousand words. For they say, that there is a great city in the midst of which, that is and still is a very strange city, or rather a country which, having fallen into the hands of a prince and his army, is still in the midst of which there stands the greatest kingdom ever seen.


‘Where?’ said the little ship. ‘Have you got some money that I am to give, or something of your own, for the bride?’


‘Have you, too, and what is your name?’ replied the little ship, ‘but I would like it out here in peace.’


‘Very well, then’ answered Mme.’s father, ‘you shall be the most valuable man in all the world.’


‘I will follow my father out into the country, and tell all you need hear about it first,’ answered Mme. ‘for, now that we have a prince, we must be careful of what he 

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("DeepESP/gpt2-spanish")

model = AutoModelForCausalLM.from_pretrained("DeepESP/gpt2-spanish")

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--DeepESP--gpt2-spanish/snapshots/1b935e39cf9893108bd2f4fb5317f48ae1c3ab5e/config.json
Model config GPT2Config {
  "_name_or_path": "DeepESP/gpt2-spanish",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "gradient_checkpointing": false,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_s

In [ ]:
generator = pipeline('text-generation', tokenizer='DeepESP/gpt2-spanish', model='DeepESP/gpt2-spanish')

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--DeepESP--gpt2-spanish/snapshots/1b935e39cf9893108bd2f4fb5317f48ae1c3ab5e/config.json
Model config GPT2Config {
  "_name_or_path": "DeepESP/gpt2-spanish",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "gradient_checkpointing": false,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_s

In [ ]:
print(generator('Había una vez', max_length=100)[0]['generated_text'])

Generate config GenerationConfig {
  "bos_token_id": 50256,
  "do_sample": true,
  "eos_token_id": 50256,
  "max_length": 50,
  "transformers_version": "4.26.1"
}

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Había una vez que el médico lo había encontrado, pero se había quedado en la cocina, para no verter demasiado. 

—Gracias. —Agnes sonrió y vio que las gotas de lluvia no se caían sobre su rostro. 

—Por cierto, ¿cuál es el problema? 

—Es mi amigo el doctor Simon —explicó él, aunque el nombre se le heló en la boca. 

—¿El que le ha dado? 

—¡Joder! No lo


In [ ]:
generator = pipeline('sentiment-analysis')

In [ ]:
print(generator('this is very good'))